# Báo cáo kỹ thuật - Autoencoder Feature Learning (CPU/GPU CUDA)

**Sinh viên:** Trương Hà Huy Tâm  
**Môn học:** CSC14120  
**Ngày:** 2025-12-16

> Notebook này vừa là báo cáo kỹ thuật, vừa là script chạy end-to-end (download CIFAR-10 → build → chạy Phase 1/2/3).

## 1. Mục tiêu dự án

Dự án yêu cầu hiện thực **pipeline học đặc trưng bằng Autoencoder** trên CIFAR-10 theo các pha tăng dần: 

- **Phase 1:** CPU baseline + data pipeline (đọc CIFAR-10 binary, forward/backward, MSE loss, training loop).
- **Phase 2:** GPU naive (CUDA kernels cơ bản, huấn luyện trên GPU).
- **Phase 3:** GPU optimized (chọn *ít nhất một* tối ưu hoá từ các category trong đề; repo này chọn **Memory Pool/Reuse Strategy** để giảm overhead cấp phát/giải phóng bộ nhớ).

Cuối cùng (Phase 4 trong đề) thường là trích xuất đặc trưng từ encoder để phục vụ bài toán phân loại (SVM). Repo hiện tại đã chuẩn bị hàm `encode()`/`save_features()` cho bước này.

## 2. Dataset và định dạng CIFAR-10 (binary)

- CIFAR-10 binary gồm 6 file: 5 file train (`data_batch_1..5`) và 1 file test (`test_batch.bin`).
- Mỗi record: **1 byte label + 3072 bytes ảnh** (32×32×3) theo thứ tự kênh (R, G, B).
- Tiền xử lý: `uint8 [0,255] → float [0,1]`.

Trong repo, phần này nằm ở `scripts/download_cifar10.py`, `include/data_loader.h`, `src/data_loader.cpp`.

In [ ]:

# (Colab/Kaggle) Chuẩn bị thư mục build và tải CIFAR-10 binary
import os, pathlib, sys, subprocess, textwrap

PROJECT_ROOT = pathlib.Path(".").resolve()
DATA_DIR = PROJECT_ROOT / "data"
BUILD_DIR = PROJECT_ROOT / "build"
BUILD_DIR.mkdir(exist_ok=True)

print("Project root:", PROJECT_ROOT)
print("Data dir:", DATA_DIR)


In [ ]:

# Download CIFAR-10 (binary) nếu chưa có
!python scripts/download_cifar10.py


## 3. Kiến trúc Autoencoder

Theo guideline trong đề, autoencoder dùng **5 lớp convolution** (encoder+decoder) với các phép toán cơ bản:

- **Conv2D 3×3** (padding, stride)
- **Add bias**
- **ReLU**
- **MaxPool 2×2** (downsample)
- **UpSampling 2×2** (nearest neighbor)
- **MSE loss** cho tái tạo ảnh

Repo triển khai hai biến thể:

- `Cpu_Autoencoder` (`include/cpu/cpu_autoencoder.h`, `src/cpu/cpu_autoencoder.cpp`)
- `Gpu_Autoencoder` + `Gpu_Autoencoder_Opt` (`include/gpu/*`, `src/gpu/*`)

> Ghi chú: `include/constants.h` đang đặt `NUM_TRAIN_SAMPLES = 20000` để giảm thời gian chạy (subset của 50k ảnh train).

## 4. Cấu trúc thư mục (repo)

```
autoencoder-feature-learning-main/
├─ include/
│  ├─ constants.h
│  ├─ data_loader.h
│  ├─ cpu/
│  │  ├─ cpu_layers.h
│  │  └─ cpu_autoencoder.h
│  └─ gpu/
│     ├─ gpu_layers.h
│     ├─ gpu_autoencoder.h
│     └─ gpu_autoencoder_opt.h
├─ src/
│  ├─ data_loader.cpp
│  ├─ main_cpu_phase_1.cpp
│  ├─ main_gpu_phase_2.cu
│  ├─ main_gpu_phase_3.cu
│  ├─ cpu/
│  │  ├─ cpu_layers.cpp
│  │  └─ cpu_autoencoder.cpp
│  └─ gpu/
│     ├─ gpu_layers.cu
│     ├─ gpu_autoencoder.cu
│     └─ gpu_autoencoder_opt.cu
└─ scripts/
   └─ download_cifar10.py
```

Notebook này sẽ build và chạy theo đúng cấu trúc trên.

## 5. Phase 1 — CPU Baseline

### 5.1 Data loader
- `load_dataset(dataset_dir, is_train)` đọc CIFAR-10 binary.
- `shuffle_dataset(dataset)` shuffle theo chỉ số mẫu.
- `create_minibatches(dataset, batch_size)` cắt batch.

### 5.2 CPU layers
Các hàm CPU trong `include/cpu/cpu_layers.h`:
- `cpu_conv2D`, `cpu_add_bias`, `cpu_relu`, `cpu_max_pooling`, `cpu_upsampling`, `cpu_mse_loss`.

### 5.3 Training loop
Trong `Cpu_Autoencoder::fit(...)`:
- epochs = 20, batch_size = 32, lr = 1e-3
- forward → loss → (backward/grad) → SGD update
- log loss và thời gian per epoch
- checkpoint/save weights (nếu bật).

In [ ]:

# Build Phase 1 (CPU)
!g++ -O2 -std=c++17 \
  -Iinclude \
  src/main_cpu_phase_1.cpp \
  src/data_loader.cpp \
  src/cpu/cpu_layers.cpp \
  src/cpu/cpu_autoencoder.cpp \
  -o build/cpu_phase1


In [ ]:

# Run Phase 1 (CPU) - truyền đường dẫn data/ nếu cần
!./build/cpu_phase1 ./data


## 6. Phase 2 — Naive GPU (CUDA)

Mục tiêu: port toàn bộ operation sang GPU với mức song song hoá cơ bản.

### 6.1 Thiết kế kernel (naive)
- **Conv2D:** mỗi thread tính 1 output pixel, đọc/ghi global memory.
- **ReLU / Bias:** in-place hoặc out-of-place theo buffer.
- **MaxPool/UpSample:** mỗi thread xử lý 1 output phần tử.
- **MSE loss:** reduction để cộng dồn sai số bình phương.
- Backward: kernel gradient cho từng layer và SGD update.

### 6.2 Training loop GPU
- Batch size trong guideline đề xuất tăng lên 64 để tận dụng GPU.
- Mỗi batch: H2D copy → forward → loss → backward → update.
- Dùng CUDA event timer để đo thời gian.

Repo: `src/main_gpu_phase_2.cu`, `include/gpu/gpu_autoencoder.h`, `src/gpu/gpu_autoencoder.cu`, `src/gpu/gpu_layers.cu`.

In [ ]:

# Build Phase 2 (Naive GPU) - yêu cầu môi trường có nvcc
!nvcc -O2 -std=c++17 \
  -Iinclude \
  src/main_gpu_phase_2.cu \
  src/data_loader.cpp \
  src/gpu/gpu_layers.cu \
  src/gpu/gpu_autoencoder.cu \
  -o build/gpu_phase2


In [ ]:

# Run Phase 2 (Naive GPU)
!./build/gpu_phase2 ./data


## 7. Phase 3 — GPU Optimized (tối ưu đã chọn)

Trong danh sách gợi ý tối ưu hoá của đề, repo này triển khai **Category 1 – Memory Pool/Reuse Strategy**:

- Thay vì `cudaMalloc/cudaFree` nhiều lần cho weights/activations/gradients, chương trình:
  1) **Tính trước tổng số float cần dùng** cho toàn bộ weights + bias + activation + gradient.
  2) `cudaMalloc` **một lần** cho `d_memory_pool`.
  3) Ánh xạ các con trỏ (`d_W1, d_b1, ..., d_grad_*`) vào các vùng con trong pool.
  4) Reuse buffers qua mọi batch/epoch.

Lợi ích chính:
- Giảm overhead cấp phát/giải phóng, đặc biệt rõ khi training lâu/ nhiều batch.
- Giảm fragment và đơn giản hoá quản lý bộ nhớ trên GPU.

File chính: `include/gpu/gpu_autoencoder_opt.h`, `src/gpu/gpu_autoencoder_opt.cu`.

> Có thể mở rộng thêm (nếu cần báo cáo nâng cao): coalescing, shared memory tiling, kernel fusion… nhưng trong repo hiện tại, điểm nhấn là **memory reuse**.

In [ ]:

# Build Phase 3 (Optimized GPU)
!nvcc -O2 -std=c++17 \
  -Iinclude \
  src/main_gpu_phase_3.cu \
  src/data_loader.cpp \
  src/gpu/gpu_layers.cu \
  src/gpu/gpu_autoencoder_opt.cu \
  -o build/gpu_phase3


In [ ]:

# Run Phase 3 (Optimized GPU)
!./build/gpu_phase3 ./data


## 8. Đánh giá & kết quả cần báo cáo

Trong phần báo cáo kỹ thuật, nên trình bày tối thiểu các mục sau (có thể copy/paste vào báo cáo nộp):

1) **Correctness check**: loss giảm theo epoch (CPU và GPU), output shape đúng.
2) **Runtime**:
   - Thời gian/epoch cho Phase 1 (CPU).
   - Thời gian/epoch cho Phase 2 (GPU naive).
   - Thời gian/epoch cho Phase 3 (GPU optimized).
3) **Speedup**:
   - Speedup(Phase2 vs Phase1), Speedup(Phase3 vs Phase2).
4) **Memory**:
   - Batch size, số parameter (tổng weights), footprint (đặc biệt Phase3 pool size in MB).

> Gợi ý: log console của `gpu_autoencoder_opt.cu` có dòng `[Optim] Allocating Memory Pool: ... MB` có thể trích vào báo cáo.


In [ ]:

# (Tuỳ chọn) Lưu log ra file để chụp vào báo cáo
!./build/gpu_phase3 ./data | tee build/phase3_log.txt


## 9. Gợi ý nội dung viết (bám sát rubric)

Bạn có thể dùng dàn ý sau để viết phần *Technical Report* (tương ứng các section trong notebook):

- **Abstract/Overview:** mục tiêu, pipeline, các phase.
- **Implementation:** data loader + layers + autoencoder + training loop.
- **CUDA Design:** mapping thread/block, memory access, reduction cho loss.
- **Optimization Chosen:** memory pool/reuse (mô tả cách tính tổng memory, ánh xạ pointer).
- **Experiments:** môi trường (GPU/CPU), hyperparams, số sample.
- **Results:** loss curve (nếu có), thời gian/epoch, speedup.
- **Limitations & Future work:** (optional) shared memory tiling, kernel fusion, mixed precision, streams.


## 10. Tài liệu tham chiếu

- *CSC14120 Final Project specification (PDF đi kèm trong repo).*  
- CIFAR-10 dataset (Krizhevsky).

> Nếu bạn cần mình xuất notebook này thành bản “báo cáo hoàn chỉnh” (điền luôn số liệu chạy thực tế từ log của bạn), bạn chỉ cần paste log output của 3 phase vào chat, mình sẽ tự động tổng hợp bảng kết quả + speedup.